### Direct links to results
[TF-MoDISco results](#tfm-results)

[Summary of motifs](#motif-summary)

[TOMTOM matches to motifs](#tomtom)

[Sample of seqlets for each motif](#seqlets)

In [ ]:
import os
import sys
sys.path.append(os.path.abspath("/users/amtseng/tfmodisco/src/"))
from tfmodisco.run_tfmodisco import import_shap_scores
from motif_bench.match_motifs import match_motifs_to_database
import modisco.visualization.viz_sequence as viz_sequence
import h5py
import numpy as np
import pandas as pd
import sklearn.cluster
import scipy.cluster.hierarchy
import collections
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import modisco
import tqdm
tqdm.tqdm_notebook()

In [ ]:
# Plotting defaults
font_manager.fontManager.ttflist.extend(
    font_manager.createFontList(
        font_manager.findSystemFonts(fontpaths="/users/amtseng/modules/fonts")
    )
)
plot_params = {
    "figure.titlesize": 22,
    "axes.titlesize": 22,
    "axes.labelsize": 20,
    "legend.fontsize": 18,
    "xtick.labelsize": 16,
    "ytick.labelsize": 16,
    "font.family": "Roboto",
    "font.weight": "bold"
}
plt.rcParams.update(plot_params)

### Define constants and paths

In [ ]:
# Define parameters/fetch arguments
tf_name = os.environ["TFM_TF_NAME"]
shap_scores_path = os.environ["TFM_SHAP_PATH"]
tfm_results_path = os.environ["TFM_TFM_PATH"]
if "TFM_TASK_INDEX" in os.environ:
    task_index = int(os.environ["TFM_TASK_INDEX"])
else:
    task_index = None
hyp_score_key = os.environ["TFM_HYP_SCORE_KEY"]

print("TF name: %s" % tf_name)
print("DeepSHAP scores path: %s" % shap_scores_path)
print("TF-MoDISco results path: %s" % tfm_results_path)
print("Task index: %s" % task_index)
print("Importance score key: %s" % hyp_score_key)

In [ ]:
# Define paths and constants
input_length = 1000
shap_score_center_size = 400

base_path = "/users/amtseng/tfmodisco/"
data_path = os.path.join(base_path, "data/processed/ENCODE/")
labels_path = os.path.join(data_path, "labels/%s" % tf_name)

# Paths to original called peaks
all_peak_beds = sorted([item for item in os.listdir(labels_path) if item.endswith(".bed.gz")])
if task_index is None:
    peak_bed_paths = [os.path.join(labels_path, item) for item in all_peak_beds]
else:
    peak_bed_paths = [os.path.join(labels_path, all_peak_beds[task_index])]

### Helper functions

In [ ]:
def extract_coords(
    seqlets_arr, one_hot_seqs, hyp_scores, pred_coords, input_length,
    input_center_cut_size
):
    """
    From the seqlets object of a TF-MoDISco pattern's seqlets and alignments,
    extracts the set of coordinates for the seqlets.
    Arguments:
        `seqlets_arr`: a TF-MoDISco pattern's seqlets object array (N-array)
        `one_hot_seqs`: an N x R x 4 array of input sequences, where R is
            the cut centered size
        `hyp_scores`: an N x R x 4 array of hypothetical importance scores
        `pred_coords`: an N x 3 object array of coordinates for the input sequences
        `input_length`: length of original input sequences, I
        `input_center_cut_size`: centered cut size of SHAP scores used
    Returns an N x Q x 4 array of one-hot seqlet sequences, an N x Q x 4 array of
    hypothetical seqlet importance scores, and an N x 3 object array of seqlet
    coordinates, where Q is the seqlet length.
    Note that it is important that the seqlet indices match exactly with the indices
    out of the N. This should be the exact sequences in the original SHAP scores.
    """
    seqlet_seqs, seqlet_hyps, seqlet_coords = [], [], []
    
    def seqlet_coord_to_input_coord(seqlet_coord):
        return seqlet_coord + ((input_length - input_center_cut_size) // 2)
        
    # For each seqlet, fetch the true/predicted profiles
    for seqlet in seqlets_arr:
        coord_index = seqlet.coor.example_idx
        seqlet_start = seqlet.coor.start
        seqlet_end = seqlet.coor.end
        seqlet_rc = seqlet.coor.is_revcomp
        
        # The one-hot-sequences and hypothetical scores are assumed to already by cut/centered,
        # so the indices match the seqlet indices
        if seqlet_rc:
            seqlet_seqs.append(np.flip(one_hot_seqs[coord_index, seqlet_start:seqlet_end], axis=(0, 1)))
            seqlet_hyps.append(np.flip(hyp_scores[coord_index, seqlet_start:seqlet_end], axis=(0, 1)))
        else:
            seqlet_seqs.append(one_hot_seqs[coord_index, seqlet_start:seqlet_end])
            seqlet_hyps.append(hyp_scores[coord_index, seqlet_start:seqlet_end])
            
        # Get the coordinates of the seqlet based on the input coordinates
        inp_start = seqlet_coord_to_input_coord(seqlet_start)
        inp_end = seqlet_coord_to_input_coord(seqlet_end)
        chrom, start, _ = pred_coords[coord_index]
        seqlet_coords.append([chrom, start + inp_start, start + inp_end])
    
    return np.stack(seqlet_seqs), np.stack(seqlet_hyps), np.array(seqlet_coords, dtype=object)

In [ ]:
def import_tfmodisco_results(tfm_results_path, hyp_scores, one_hot_seqs, input_center_cut_size):
    """
    Imports the TF-MoDISco results object.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `hyp_scores`: hypothetical importance scores used for this run
        `one_hot_seqs`: input sequences used for this run
        `input_center_cut_size`: centered cut size of SHAP scores used
    """ 
    # Everything should already be cut to `input_center_cut_size`
    act_scores = hyp_scores * one_hot_seqs
    
    track_set = modisco.tfmodisco_workflow.workflow.prep_track_set(
        task_names=["task0"],
        contrib_scores={"task0": act_scores},
        hypothetical_contribs={"task0": hyp_scores},
        one_hot=one_hot_seqs
    )
    
    with h5py.File(tfm_results_path,"r") as f:
        return modisco.tfmodisco_workflow.workflow.TfModiscoResults.from_hdf5(f, track_set=track_set)

In [ ]:
def import_peak_table(peak_bed_paths):
    tables = []
    for peak_bed_path in peak_bed_paths:
        table = pd.read_csv(
            peak_bed_path, sep="\t", header=None,  # Infer compression
            names=[
                "chrom", "peak_start", "peak_end", "name", "score",
                "strand", "signal", "pval", "qval", "summit_offset"
            ]
        )
        # Add summit location column
        table["summit"] = table["peak_start"] + table["summit_offset"]
        tables.append(table)
    return pd.concat(tables)

In [ ]:
def get_summit_distances(coords, peak_table):
    """
    Given a set of coordinates, computes the distance of the center of each
    coordinate to the nearest summit.
    Arguments:
        `coords`: an N x 3 object array of coordinates
        `peak_table`: a 6-column table of peak data, as imported by
            `import_peak_table`
    Returns and N-array of integers, which is the distance of each coordinate
    midpoint to the nearest coordinate.
    """
    chroms = coords[:, 0]
    midpoints = (coords[:, 1] + coords[:, 2]) // 2
    dists = []
    for i in range(len(coords)):
        chrom = chroms[i]
        midpoint = midpoints[i]
        rows = peak_table[peak_table["chrom"] == chrom]
        dist_arr = (midpoint - rows["summit"]).values
        min_dist = dist_arr[np.argmin(np.abs(dist_arr))]
        dists.append(min_dist)
    return np.array(dists)

In [ ]:
def plot_summit_dists(summit_dists):
    """
    Plots the distribution of seqlet distances to summits.
    Arguments:
        `summit_dists`: the array of distances as returned by
            `get_summit_distances`
    """
    plt.figure(figsize=(8, 6))
    num_bins = max(len(summit_dists) // 30, 20)
    plt.hist(summit_dists, bins=num_bins, color="purple")
    plt.title("Histogram of distance of seqlets to peak summits")
    plt.xlabel("Signed distance from seqlet center to nearest peak summit (bp)")
    plt.show()

In [ ]:
background_freqs = np.array([0.25, 0.25, 0.25, 0.25])
def info_content(track, pseudocount=0.001):
    """
    Given an L x 4 track, computes information content for each base and
    returns it as an L-array.
    """
    num_bases = track.shape[1]
    # Normalize track to probabilities along base axis
    track_norm = (track + pseudocount) / (np.sum(track, axis=1, keepdims=True) + (num_bases * pseudocount))
    ic = track_norm * np.log2(track_norm / np.expand_dims(background_freqs, axis=0))
    return np.sum(ic, axis=1)

In [ ]:
def pfm_to_pwm(pfm):
    ic = info_content(pfm)
    return pfm * np.expand_dims(ic, axis=1)

In [ ]:
def dna_to_one_hot(dna_seq):
    """
    Converts a single DNA sequence as a string to a one-hot encoding.
    """
    dna_seq = dna_seq.upper()
    one_hot_map = np.identity(5)[:, :-1]
    base_vals = np.frombuffer(bytearray(dna_seq, "utf8"), dtype=np.int8)
    base_vals[~np.isin(base_vals, np.array([65, 67, 71, 84]))] = 85
    _, base_inds = np.unique(base_vals, return_inverse=True)
    return one_hot_map[base_inds].reshape((len(dna_seq), 4))

### Import SHAP scores, profile predictions, and TF-MoDISco results

In [ ]:
# Import SHAP coordinates and one-hot sequences
hyp_scores, _, one_hot_seqs, shap_coords = import_shap_scores(shap_scores_path, hyp_score_key, center_cut_size=shap_score_center_size)
# This cuts the sequences/scores off just as how TF-MoDISco saw them, but the coordinates are uncut

In [ ]:
# Import the set of peaks
peak_table = import_peak_table(peak_bed_paths)

In [ ]:
# Import the TF-MoDISco results object
tfm_obj = import_tfmodisco_results(tfm_results_path, hyp_scores, one_hot_seqs, shap_score_center_size)

### Plot some SHAP score tracks
Plot the central region of some randomly selected actual importance scores

In [ ]:
for index in np.random.choice(hyp_scores.shape[0], size=5, replace=False):
    viz_sequence.plot_weights((hyp_scores[index] * one_hot_seqs[index])[100:300], subticks_frequency=100)

<a id="tfm-results"></a>
### Plot TF-MoDISco results
Plot all motifs by metacluster

In [ ]:
motifs = []  # Save the motifs as trimmed CWMs
motif_pfms = []  # Save the trimmed PFMs, as well
num_seqlets = []  # Number of seqlets for each motif
motif_seqlets = {}  # Save seqlets of each motif
metaclusters = tfm_obj.metacluster_idx_to_submetacluster_results
num_metaclusters = len(metaclusters.keys())
for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
    metacluster = metaclusters[metacluster_key]
    print("Metacluster %d/%d" % (metacluster_i + 1, num_metaclusters))
    print("==========================================")
    patterns = metacluster.seqlets_to_patterns_result.patterns
    if not patterns:
        break
    motifs.append([])
    motif_pfms.append([])
    num_seqlets.append([])
    num_patterns = len(patterns)
    for pattern_i, pattern in enumerate(patterns):
        seqlets = pattern.seqlets
        print("Pattern %d/%d" % (pattern_i + 1, num_patterns))
        print("--------------------------------------")
        
        pfm = pattern["sequence"].fwd
        hcwm = pattern["task0_hypothetical_contribs"].fwd
        cwm = pattern["task0_contrib_scores"].fwd
        print("%d seqlets" % len(seqlets))
        print("Sequence")
        viz_sequence.plot_weights(pfm, subticks_frequency=10)
        print("Hypothetical contributions")
        viz_sequence.plot_weights(hcwm, subticks_frequency=10)
        print("Actual contributions (CWM)")
        viz_sequence.plot_weights(cwm, subticks_frequency=10)
        
        # Trim motif based on IC
        pfm_ic = info_content(pfm)
        ic_trim_thresh = np.max(pfm_ic) * 0.2  # Cut off anything less than 20% of max IC
        pass_inds = np.where(pfm_ic >= ic_trim_thresh)[0]
        trimmed_cwm = cwm[np.min(pass_inds): np.max(pass_inds) + 1]
        trimmed_pfm = pfm[np.min(pass_inds): np.max(pass_inds) + 1]
        motifs[-1].append(trimmed_cwm)
        motif_pfms[-1].append(trimmed_pfm)
        
        num_seqlets[-1].append(len(seqlets))
        
        seqlet_seqs, seqlet_hyps, seqlet_coords = extract_coords(
            seqlets, one_hot_seqs, hyp_scores, shap_coords, input_length,
            shap_score_center_size
        )
        
        key_pair = (metacluster_key, pattern_i)
        motif_seqlets[key_pair] = (seqlet_seqs, seqlet_hyps)

        assert np.allclose(np.sum(seqlet_seqs, axis=0) / len(seqlet_seqs), pattern["sequence"].fwd)
        # ^Sanity check: PFM derived from seqlets match the PFM stored in the pattern
        
        print("Seqlet distance from summits")
        summit_dists = get_summit_distances(seqlet_coords, peak_table)
        plot_summit_dists(summit_dists)

<a id="motif-summary"></a>
**Summary of motifs**

Motifs are trimmed based on information content, and presented in descending order by number of supporting seqlets. The motifs are separated by metacluster.

In [ ]:
for i in range(len(motifs)):
    print("Metacluster %d/%d" % (i + 1, num_metaclusters))
    print("==========================================")
    for j in range(len(motifs[i])):
        print("Motif %d/%d (number of seqlets: %d)" % (j + 1, len(motifs[i]), num_seqlets[i][j]))
        viz_sequence.plot_weights(motifs[i][j])

<a id="tomtom"></a>
**Top TOMTOM matches for each motif**

Here, the motifs are plotted as PWMs

In [ ]:
for i in range(len(motifs)):
    print("Metacluster %d/%d" % (i + 1, num_metaclusters))
    print("==========================================")
    tomtom_matches = match_motifs_to_database(motif_pfms[i])
    for j in range(len(motifs[i])):
        print("Motif %d/%d (number of seqlets: %d)" % (j + 1, len(motifs[i]), num_seqlets[i][j]))
        viz_sequence.plot_weights(pfm_to_pwm(motif_pfms[i][j]))
        print("Top TOMTOM matches")
        for match_name, match_seq, match_qval in tomtom_matches[j]:
            print("%s: q-val = %f" % (match_name, match_qval))
            viz_sequence.plot_weights(dna_to_one_hot(match_seq))
        print("------------------------------------------")

<a id="seqlets"></a>
**Sample of seqlets supporting each motif**

In [ ]:
num_seqlets_to_show = 10
metaclusters = tfm_obj.metacluster_idx_to_submetacluster_results
num_metaclusters = len(metaclusters.keys())
for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
    metacluster = metaclusters[metacluster_key]
    print("Metacluster %d/%d" % (metacluster_i + 1, num_metaclusters))
    print("==========================================")
    patterns = metacluster.seqlets_to_patterns_result.patterns
    if not patterns:
        break
    num_patterns = len(patterns)
    for pattern_i, pattern in enumerate(patterns):
        seqlets = pattern.seqlets
        print("Pattern %d/%d" % (pattern_i + 1, num_patterns))
        print("--------------------------------------")
        print("Actual contributions (CWM)")
        viz_sequence.plot_weights(pattern["task0_contrib_scores"].fwd, subticks_frequency=10)
        
        key_pair = (metacluster_key, pattern_i)
        seqlet_seqs, seqlet_hyps = motif_seqlets[key_pair]
        
        print("Sample of seqlets")
        sample_size = min(num_seqlets_to_show, len(seqlet_seqs))
        sample_inds = np.random.choice(len(seqlet_seqs), size=sample_size, replace=False)
        for i in sample_inds:
            viz_sequence.plot_weights(seqlet_seqs[i] * seqlet_hyps[i], subticks_frequency=10)